# Blackjack with AI player

In [444]:
import random

In [492]:
RANKS = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

# Card class keeps:
    # rank - The rank of the card [2,3,4,5,6,7,8,9,10,J,Q,K,A]
    # value - The point value associated with the rank
    # isAce - True if the card is an Ace
class Card(object):
    
    def __init__(self, rank = None):
        self.rank = rank
        
        if not rank: # null card
            self.value = 0
            self.isAce = False
            return
        elif rank == 'A':
            self.value = 11
            self.isAce = True
        elif rank in ['J','Q','K']:
            self.value = 10
            self.isAce = False
        else:
            self.value = int(rank) # 2-10
            self.isAce = False
         
    def __str__(self):
        return str(self.rank)
    
    def __eq__(self, card):
        return self.value == card.value


# Deck class keeps:
    # cards - list of cards remaining in the deck
class Deck(object):
    
    def __init__(self, nDecks = 1):
        self.cards = [Card()]
        self.buildDeck(nDecks)
        self.prepareDeck()
    
    def buildDeck(self, nDecks):
        for i in range(nDecks):
            for j in range(4): # 4 of each card per deck
                for rank in RANKS:
                    self.cards.append(Card(rank))
    
    def prepareDeck(self):
        for i in range(7):
            self.shuffle()
            
    def shuffle(self):
        random.shuffle(self.cards)
        
    def draw(self):
        card = self.cards.pop(0)
        return card
    
    def __len__(self):
        return len(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])
    
# Hand class keeps
    # hand - cards in the hand
    # bet - the bet placed on the hand
class Hand(object):
    
    def __init__(self, card = None, bet = 50):
        self.bet = bet
        self.cards = [Card()]
        if card:
            self.cards.append(card)
        
    def addCard(self, card):
        self.cards.append(card)     
            
    def removeCard(self):
        return self.cards.pop()
    
    def checkBust(self):
        if self.value > 21:
            return True
        return False
        
    @property
    def value(self):
        value = sum(card.value for card in self.cards)
        aces = sum(card.isAce for card in self.cards)  
        # Aces are initially given a value of 11, they are ammended to 1 if the total valaue is > 21
        while (value > 21) and aces: 
            value -= 10
            aces -= 1
        return value
    
    @property
    def checkSoft(self):
        value = sum(card.value for card in self.cards)
        aces = sum(card.isAce for card in self.cards)  
        # Aces are initially given a value of 11, they are ammended to 1 if the total valaue is > 21
        while (value > 21) and aces: 
            value -= 10
            aces -= 1
        return aces
        
    def __str__(self):
        return str([str(card) for card in self.cards])
        

#### Testing Card, Deck, and Hand

In [494]:
ace = Card('A')
king = Card('K')
queen = Card('Q')
four = Card('4')
nine = Card('9')
noCard = Card()
passed = 0
if ace.value == 11:
    if ace.rank == 'A':
        passed += 1
        print ('ace: passed')
if king.value == 10:
    if king.rank == 'K':
        passed += 1
        print ('king: passed')
if queen.value == 10:
    if queen.rank == 'Q':
        passed += 1
        print ('queen: passed')
if four.value == 4:
    if four.rank == '4':
        passed += 1
        print ('four: passed')
if nine.value == 9:
    if nine.rank == '9':
        passed += 1
        print ('nine: passed')
if noCard.value == 0:
        passed += 1
        print ('noCard: passed')
if king == queen:
    print('Equality passed')
    passed += 1 
if not king == nine:
    print('Not Equality passed')
    passed += 1
if passed == 8:
    print('All tests passed')
else:
    print(str(6-passed) + ' tests failed')

ace: passed
king: passed
queen: passed
four: passed
nine: passed
noCard: passed
Equality passed
Not Equality passed
All tests passed


In [446]:
# Player class keeps:
    # money
    # hands
class Player:
    
    def __init__(self, money = 5000, bet = 50):
        self.money = money
        self.betAmount = bet
        self.bets = []
        self.hands = [Hand()]
        
    def makeBet(self, handNumber):
        if handNumber >= len(self.bets): # new hand gets a new bet
            self.bets.append(self.betAmount)
        else: # double the bet
            self.bets[handNumber] += self.betAmount
        # remove the bet amount from players money
        self.money -= self.betAmount
            
    # True if player has enough money to make a bet, False otherwise
    def hasMoney(self):
        return self.money > self.betAmount
        
    def hit(self, handNumber, card):
        self.hands[handNumber].addCard(card)

    def doubleDown(self, handNumber, card):
        self.hit(handNumber, card)
        self.makeBet(handNumber)

    def split(self, handNumber, cards):
        newHand = Hand()
        newHand.addCard(self.hands[handNumber].removeCard()) # remove one card from the hand and add to newHand
        self.hands[handNumber].addCard(cards[0]) # add a card to the old hand
        self.hands.append(newHand.addCard(cards[1])) # add new hand to players hands w/ second card
        self.makeBet(len(self.hands)) # add a bet to the new hand
        
    def showHands(self):
        print("Player has %d hands: \n" % len(self.hands))
        for hand in self.hands:
            print(hand)
        print()
        
    def getHand(self, handNumber):
        return self.hands[handNumber]
    
    def stats(self):
        print("Player has $%d left." % self.money)
        # possible other stats later on
        

# Dealer class keeps:
    # deck
    # hand
    # isBust
class Dealer:
    
    def __init__(self, deck, Player):
        self.deck = deck
        self.hand = Hand()
        self.Player = Player
    
    def deal(self):
        for i in range(2): # deal each player, including the dealer, 2 cards
            self.hand.addCard(self.deck.draw()) # Deal the dealer a card
            if i == 0: # dealers first card is face down
                self.hand.cards[0].hideCard() 
            self.Player.hands[0].addCard(self.deck.draw()) # Deal the player a card   
        self.Player.makeBet(0) # player makes a bet on hand
       
    def play(self):
        while self.hand.value <= 17: # hits until they have 17
            # if soft 17, dealer must hit
            if self.hand.value == 17:
                if self.hand.checkSoft:
                    self.hit()
            # less than 17, must hit
            else:
                self.hit()
            
            
    def hit(self):
        self.hand.addCard(self.deck.draw())
            
    def stand(self):
        return self.hand
    
    def showHands(self):
        print('Dealers Hand:')
        print(self.hand)
    
    
def validMoves():
    return 0

def makeMoves():
    return 0

In [447]:
class Blackjack:
    
    def __init__(self, Player):
        self.Player = Player
        self.Deck = Deck(6)
        self.Dealer = Dealer(self.Deck, Player)
        self.Dealer.deal()
        
    def newHand(self):
        self.Player.hands = [Hand()]
        self.Dealer.hand = Hand()
        self.Dealer.deal()
        
    def newDeck(self):
        self.Deck = Deck(6)
       
    # hand is player's hand
    def gameStatus(self, hand):
        # player bust
        if hand.checkBust():
            return -1
        # dealer bust
        if self.Dealer.hand.checkBust():
            return 1
        # draw
        if hand.value == self.Dealer.hand.value:
            return 0
        # player win
        if hand.value > self.Dealer.hand.value:
            return 1
        # dealer win
        if hand.value < self.Dealer.hand.value:
            return -1
        

In [448]:
# returns a list of valid moves for a hand
def validMoves(player, handNumber):
    moves = ['hit','stand']
    # check that hand has 2 cards and the player has money to make a bet
    if len(player.hands[handNumber].cards) == 2:
        if player.hasMoney(): 
            moves.append('doubleDown')
            if player.hands[handNumber].cards[0][1] == player.hands[handNumber].cards[1][1]: # compare ranks
                moves.append('split')
            
    return moves
    
def makeMove(game, handNumber, move):
    if move == 'hit':
        game.player.hit(handNumber, game.deck.draw())
        return False
    elif move == 'split':
        game.player.split(handNumber, [game.deck.draw(), game.deck.draw()])
        return False
    elif move == 'doubleDown':
        game.player.doubleDown(handNumber, game.deck.draw())
        return True
    else:
        return True
        


In [449]:
import random

# determines if a greedy move should be taken
def epsilonGreedy (epsilon, Q, player, handNumber, validMovesF):
    validMoves = validMovesF(player, handNumber)
    
    if np.random.uniform() < epsilon: # Random choice
        return random.choice(validMoves)
    else: # Greedy choice
        # Please Stand By - In Progress
        return 0

In [450]:
import numpy as np

def trainQ(nRepetitions, learningRate, epsilonDecayRate, validMovesF, makeMoveF):
    epsilon = 1.0
    
    Q = {}
    outcomes = np.zeros(nRepetitions)
    epsilons = np.zeros(nRepetitions)
    
    for gameNum in range(nRepetitions):
        epsilon *= epsilonDecayRate  # decay epsilon to move away from random choices
        epsilons[gameNum] = epsilon
        
        # create a game
        player = Player()
        blackjack = Blackjack(player)
        
        done = False
        
        # play some blackjack
        while len(blackjack.Deck) >  52: # deck is 6 decks, last deck is a buffer deck
            if blackjack.player.hasMoney(): # make sure player has the funds to play
                blackjack.newHand() # deal a hand
                handNumber = 0
            
                # For each players hand
                while handNumber < len(blackjack.player.hands):
                    done = False
                
                    # Player plays until they stand or bust
                    while not done:
                        # Determine a move for the players hand
                        move = epsilonGreedy(epsilon, Q, blackjack.player, handNumber, validMovesF)
                        
                        # make a move on the hand, done if double down or stand
                        done = makeMoveF(blackjack, handNumber, move)
                        
                        # check if bust
                        if blackjack.player.hands[handNumber].checkBust():
                            done = True
                            
                    handNumber += 1 # next hand
            else:
                break # player doesn't have the funds, game over 
        
        ### After Player plays all his hands dealer plays
        ### After Dealer plays all player hands are checked
            # payout on wins
            
        
    
    # need to add Q table stuff
        # whats my key? my cards or total value of cards?
        # how will i keep track of hands?
            # list of moves
            # each hand that leads to a win or loss or draw gets a value
    # should valid moves contain a blackjack?
        # or should it learn to stand on blackjack
        # i think blackjack is an automatic win
        # where should i check for blackjack
    # greedy choice
    # payout on wins (game)
    
    # test that the bets are running correctly - deal, split, doubleDown
        # check that bets are correct and money is updated properly
    # re-test valid moves
        # updated to include player has the funds required

## Testing

In [451]:
deck = Deck(1)
print(len(deck))
print(str(deck))
print(str(deck.draw()))
print(str(deck.draw()))
print(len(deck))

52
["('club', '4')", "('spade', '10')", "('diamond', '7')", "('diamond', 'J')", "('diamond', '2')", "('spade', '2')", "('heart', '9')", "('heart', 'A')", "('heart', 'J')", "('spade', '9')", "('spade', 'A')", "('diamond', '6')", "('heart', 'K')", "('diamond', '3')", "('heart', '2')", "('club', 'J')", "('diamond', '10')", "('club', 'Q')", "('club', '6')", "('club', '7')", "('heart', '8')", "('heart', '4')", "('heart', '10')", "('diamond', '8')", "('club', '3')", "('club', '2')", "('spade', 'J')", "('diamond', 'Q')", "('club', '9')", "('spade', '5')", "('club', 'K')", "('spade', '8')", "('spade', 'K')", "('club', '5')", "('diamond', '9')", "('spade', '7')", "('heart', '3')", "('heart', 'Q')", "('spade', 'Q')", "('diamond', '4')", "('diamond', '5')", "('spade', '4')", "('diamond', 'K')", "('heart', '7')", "('spade', '6')", "('diamond', 'A')", "('club', '10')", "('club', 'A')", "('spade', '3')", "('heart', '6')", "('club', '8')", "('heart', '5')"]
('club', '4')
('spade', '10')
50


In [452]:
deck = Deck(3)
print(len(deck))
hand = Hand()
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
print(str(hand.value))

156
["('club', '10')"]
["('club', '10')", "('spade', 'A')"]
["('club', '10')", "('spade', 'A')", "('heart', '8')"]
19


In [453]:
player = Player()
game = Blackjack(player)
print(len(game.Deck))
game.Player.showHands()
game.Dealer.showHands()
game.Dealer.hand.cards[0].showCard()
game.Dealer.showHands()
game.newHand()
print(len(game.Deck))
game.Player.showHands()
game.Dealer.hand.cards[0].showCard()
game.Dealer.hit()
game.Dealer.showHands()
game.newDeck()
print(len(game.Deck))

308
Player has 1 hands: 

["('heart', '8')", "('club', '6')"]

Dealers Hand:
["('X',)", "('club', '2')"]
Dealers Hand:
["('club', '6')", "('club', '2')"]
304
Player has 1 hands: 

["('spade', '8')", "('spade', '8')"]

Dealers Hand:
["('heart', '9')", "('diamond', '6')", "('diamond', '5')"]
312


In [454]:
game.gameStatus(game.Player.hands[0])

-1

#### Testing validMoves and makeMove

In [455]:
# Testing valid Moves
player = Player()
player.hands[0].addCard(('diamond', 'K'))
player.hands[0].addCard(('heart', 'K'))

# check all moves are avilable
moves = validMoves(player, 0)
if moves == ['hit','stand','doubleDown','split']:
    print('All moves valid: passed')
else:
    print('All moves valid: failed')
    print(moves)
    
# check no moves
player = Player(0)
player.hands[0].addCard(('diamond', 'K'))
player.hands[0].addCard(('heart', 'K'))
moves = validMoves(player, 0)
if moves == ['hit','stand']:
    print('Basic moves valid: passed')
else:
    print('Basic moves valid: failed')
    print(moves)
    
# check no split moves
player = Player()
player.hands[0].addCard(('diamond', 'K'))
player.hands[0].addCard(('heart', '5'))
moves = validMoves(player, 0)
if moves == ['hit','stand', 'doubleDown']:
    print('No split moves valid: passed')
else:
    print('No split moves valid: failed')
    print(moves)
    
# check split moves
player = Player()
player.hands[0].addCard(('diamond', '5'))
player.hands[0].addCard(('heart', '5'))
moves = validMoves(player, 0)
if moves == ['hit','stand', 'doubleDown', 'split']:
    print('Split moves valid: passed')
else:
    print('Split moves valid: failed')
    print(moves)

All moves valid: passed
Basic moves valid: passed
No split moves valid: passed
Split moves valid: passed


In [461]:
player = Player()
player.hands[0].addCard(('diamond', 'K'))
player.hands[0].addCard(('heart', 'K'))
player.split(0, [('heart', '5'), ('diamond', '5')])
print(player.getHand(0))
print(player.getHand(1))
testHands = []
testHands.append(tuple(('diamond', 'K'), ('heart', '5')))
if player.hands[0] == testHands[0]:   
    if player.hands[1] == ["('heart', 'K')", "('diamond', '5')"]:
        print('split move: passed')
    else:
        print('split move: failed')
else:
    print('split move: failed')

["('diamond', 'K')", "('heart', '5')"]
None


TypeError: tuple() takes at most 1 argument (2 given)